In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import xarray as xr
from glob import glob

In [3]:
path = "/dccstor/sallesd1/datasets/hurricanes/MERRA2.const_2d_ctm_Nx.00000000"
ds = xr.open_dataset(path + ".nc4")

In [4]:
ds_small = ds.sel(lat=slice(30, 45), lon=slice(-100, -70))
da = ds_small[list(ds_small.data_vars)[0]]

In [ ]:
lat = da.lat.values
lon = da.lon.values
lon2d, lat2d = np.meshgrid(lon, lat)
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
ax.coastlines()
ax.gridlines(draw_labels=True)
if len(da.values.shape) == 4:
    mesh = ax.pcolormesh(lon2d, lat2d, da.values[0, 0, :, :], transform=ccrs.PlateCarree(), cmap='viridis')
elif len(da.values.shape) == 3:
    mesh = ax.pcolormesh(lon2d, lat2d, da.values[0, :, :], transform=ccrs.PlateCarree(), cmap='viridis')
else:
    raise ValueError()
ax.add_feature(cfeature.LAND, facecolor='lightgray')
plt.show()

In [ ]:
encoding = {cvar: {"zlib": True, "complevel": 9} for cvar in ds_small.data_vars}
ds_small.to_netcdf(path + "SMALL.nc4", mode="w", encoding=encoding)

In [ ]:
ds_new = xr.open_dataset(path + "SMALL.nc4")
err = ds_new - ds_small
for var in err.data_vars:
    print(f"{var}:\t{err[var].sum().values}")

In [16]:
gt = xr.open_dataset("/u/dchevita/projects/Prithvi-WxC/test/raw_data/MERRA_pres_20200330.gt")
ds = xr.open_dataset("/u/dchevita/projects/Prithvi-WxC/test/raw_data/MERRA2_400.inst3_3d_asm_Nv.20200330.nc4")
for var in gt.data_vars:
    assert (gt[var]-ds[var]).sum().values == 0.0, f"Error in variable: {var}"